In [1]:
import sys
print(sys.version)

3.7.1 | packaged by conda-forge | (default, Feb 26 2019, 04:46:14) [MSC v.1900 64 bit (AMD64)]


In [2]:
import random
import gym
import keras
print(keras.__version__)
print(keras.backend)
import numpy as np
import math

Using Theano backend.
WARNING (theano.configdefaults): install mkl with `conda install mkl-service`: No module named 'mkl'
WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


2.2.4
<module 'keras.backend' from 'C:\\Users\\risha\\.conda\\envs\\envfresh3.7\\lib\\site-packages\\keras\\backend\\__init__.py'>


In [3]:
from collections import deque

In [5]:
env=gym.make('CartPole-v0')

for i_episode in range(20):
    observation=env.reset()
    for t in range(100):
        env.render()
        # print(observation)
        action=env.action_space.sample()
        observation,reward,done,info=env.step(action)

C:\Users\risha\.conda\envs\envfresh3.7\lib\site-packages\gym\logger.py:30: UserWarning: WARN: You are calling 'step()' even though this environment has already returned done = True. You should always call 'reset()' once you receive 'done = True' -- any further steps are undefined behavior.
  warnings.warn(colorize('%s: %s'%('WARN', msg % args), 'yellow'))


In [6]:
print(env.action_space)
print(env.observation_space)

Discrete(2)
Box(4,)


In [9]:
#defining training Parameters

n_episodes=1000
n_win_ticks=195
max_env_steps= None

gamma=1.0  #(Discount Factor)

epsilon=1.0  #(Exploration Factor)
epsilon_min= 0.01
epsilon_decay=0.995   #for how quickly it is going to stop exploring

alpha=0.01  #Learning Rate (i.i to what exten the new written info is going to override the new information)
alpha_decay=0.01

batch_size=64 #batch size of 64 samples

#for openaigym
monitor= False
quiet= False

#Environment Parameters

memory=deque(maxlen=100000)
env=gym.make('CartPole-v0')
if max_env_steps is not None:
    env.max_episode_steps=max_env_steps
    


In [14]:
#Building Neural network Here

from keras.models import Sequential
from keras.layers import Dense
from keras.optimizers import Adam


# model definition

Model= Sequential()
Model.add(Dense(24,input_dim=4,activation='relu'))
Model.add(Dense(48,activation='relu'))
Model.add(Dense(2,activation='relu'))
Model.compile(loss='mse',optimizer=Adam(lr=alpha,decay=alpah_decay))

ImportError: DLL load failed: The specified procedure could not be found.

In [ ]:
# defining necessary functions


def remember (state, action, reward,next_state, done):
    memory.append((state,action,reward,next_state,done))
    
def choose_action(state,epsilon):
    return env.action_space.sample() if(np.random()<= epsilon) else np.argmax(model.predict(state))

def get_epsilon(t):
    return max(epsilon_min, min(epsilon,1.0- math.log10((t+1)*epsilon_decay)))

def preprocess_state(state):
    return np.reshape(state,[1,4])

def replay(batch_size,epsilon):
    x_batch, y_batch=[],[]
    minibatch=random.sample(memory,min(len(memory),batch_size))
    
    for state, action, reward, next_state,done in minibatch:
        y_target= model.predict(state)
        y_target[0][action]=reward if done else reward +gamma * np.max(model.predict(next_state)[0])
        x_batch.append(state[0])
        y_batch.append(y_target[0])
        
    model.fit(np.array(x_batch),np.array(y_batch),batch_size=len(x_batch),verbose=0)
    
    if epsilon> epsilon_min:
        
        epsilon*=epsilon_decay

In [ ]:
def run():
    scores=deque(maxlen=100)
    
    for e in range (n_episodes):
        satte=preprocess_state(env.reset())
        done=False
        i=0
        while not done:
            action= choose_action(state,get_epsilon(e))
            next_state,reward,done, _ = env.step(action)
            env.render()
            next_state=preprocess_state(next_state)
            remember(state,action,reward,next_state,done)
            state=next_state
            i+=1
            scores.append(i)
            mean_score=np.mean(scores)
            
            if mean_score>=n_win_ticks and e>=100:
                if not quiet: print ()